# memstack-agent-ui React Hooks

本 notebook 演示 @memstack-agent-ui/react 包的 React Hooks。

## 概述

提供的 Hooks：
- **useAgentChat** - 主要聊天 hook，类似 Vercel AI SDK
- **useConversation** - 访问会话状态
- **useStreaming** - 访问流式内容
- **useAgentContext** - 访问 Agent Provider 上下文

## 1. 安装和设置

In [ ]:
# 安装

install_example = '''
# npm
npm install @memstack-agent-ui/react @memstack-agent-ui/sdk @memstack-agent-ui/core

# pnpm
pnpm add @memstack-agent-ui/react @memstack-agent-ui/sdk @memstack-agent-ui/core
'''

print(install_example)

## 2. AgentProvider 设置

In [ ]:
# 首先用 AgentProvider 包装应用

provider_setup = '''
import { AgentProvider } from '@memstack-agent-ui/react';

function App() {
  return (
    <AgentProvider
      wsUrl="ws://localhost:8000/agent/ws"
      token="your-auth-token"
      conversationId="conv-123"
      wsClientOptions={{
        heartbeatInterval: 30000,
        maxReconnectAttempts: 5,
      }}
    >
      <ChatInterface />
    </AgentProvider>
  );
}
'''

print(provider_setup)

## 3. useAgentChat Hook

In [ ]:
# 主要的聊天 hook

use_agent_chat = '''
import { useAgentChat } from '@memstack-agent-ui/react';

function ChatInput() {
  const { submit, isRunning, status, error } = useAgentChat({
    wsUrl: 'ws://localhost:8000/agent/ws',
    token: 'your-token',
    conversationId: 'conv-123',
    
    // 回调
    onStatusChange: (status) => console.log('Status:', status),
    onError: (error) => console.error('Error:', error),
    onComplete: (data) => console.log('Complete:', data),
    onAgentError: (data) => console.error('Agent error:', data),
    onMessageAcknowledged: (data) => console.log('Message ack:', data),
  });

  const [input, setInput] = useState('');

  const handleSubmit = () => {
    if (input.trim() && !isRunning) {
      submit(input.trim());
      setInput('');
    }
  };

  return (
    <div>
      <input
        value={input}
        onChange={(e) => setInput(e.target.value)}
        onKeyDown={(e) => e.key === 'Enter' && handleSubmit()}
        disabled={isRunning}
        placeholder="Type a message..."
      />
      <button onClick={handleSubmit} disabled={isRunning || !input.trim()}>
        {isRunning ? 'Running...' : 'Send'}
      </button>
      
      {error && <div className="error">{error.message}</div>}
      <div className="status">Status: {status}</div>
    </div>
  );
}
'''

print(use_agent_chat)

## 4. useAgentChat 返回值

In [ ]:
# useAgentChat 返回的接口

return_interface = '''
interface UseAgentChatReturn {
  /** 提交消息到 Agent */
  submit: (content: string, fileId?: string[]) => void;

  /** Agent 是否正在运行 */
  isRunning: boolean;

  /** 当前 WebSocket 连接状态 */
  status: WebSocketStatus;

  /** 当前错误（如果有） */
  error: Error | null;

  /** WebSocket 客户端实例（高级用法） */
  client: WebSocketClient | null;
}
'''

print(return_interface)

## 5. useConversation Hook

In [ ]:
# 访问会话状态

use_conversation = '''
import { useConversation } from '@memstack-agent-ui/react';

function ConversationView({ conversationId }: { conversationId: string }) {
  const {
    timeline,
    messages,
    agentState,
    streamStatus,
    isStreaming,
    currentThought,
    streamingThought,
    activeToolCalls,
    pendingToolsStack,
    tasks,
    costTracking,
    suggestions,
    pendingHITLSummary,
    isPlanMode,
    error,
  } = useConversation({
    conversationId,
    includeTimeline: true,
    includeMessages: true,
    includeAgentState: true,
  });

  return (
    <div>
      {/* 状态栏 */}
      <StatusBar
        agentState={agentState}
        isStreaming={isStreaming}
        costTracking={costTracking}
      />

      {/* 消息列表 */}
      <MessageList messages={messages} />

      {/* 执行时间线 */}
      <ExecutionTimeline timeline={timeline} />

      {/* 当前思考 */}
      {currentThought && (
        <ThinkingBubble content={currentThought} />
      )}

      {/* 活动工具调用 */}
      {activeToolCalls.size > 0 && (
        <ActiveTools tools={activeToolCalls} />
      )}

      {/* 任务列表 */}
      {tasks.length > 0 && <TaskList tasks={tasks} />}

      {/* 建议 */}
      {suggestions.length > 0 && (
        <SuggestionsList suggestions={suggestions} />
      )}

      {/* HITL 请求 */}
      {pendingHITLSummary && (
        <HITLRequest summary={pendingHITLSummary} />
      )}

      {/* 错误 */}
      {error && <ErrorMessage message={error} />}
    </div>
  );
}
'''

print(use_conversation)

## 6. useStreaming Hook

In [ ]:
# 专注于流式状态

use_streaming = '''
import { useStreaming } from '@memstack-agent-ui/react';

function StreamingContent({ conversationId }: { conversationId: string }) {
  const {
    streamStatus,
    isStreaming,
    streamingAssistantContent,
    currentThought,
    streamingThought,
    isThinkingStreaming,
    activeToolCalls,
    pendingToolsStack,
  } = useStreaming({
    conversationId,
    includeAssistantContent: true,
    includeThoughtContent: true,
    includeToolCalls: true,
  });

  return (
    <div className="streaming-content">
      {/* 流式指示器 */}
      {isStreaming && (
        <div className="streaming-indicator">
          <Spinner />
          <span>Agent is thinking...</span>
        </div>
      )}

      {/* 流式思考内容 */}
      {isThinkingStreaming && streamingThought && (
        <div className="thinking-bubble">
          <span className="icon">💭</span>
          <span>{streamingThought}</span>
        </div>
      )}

      {/* 流式助手内容 */}
      {streamingAssistantContent && (
        <div className="assistant-content">
          <Markdown content={streamingAssistantContent} />
        </div>
      )}

      {/* 活动工具调用 */}
      {activeToolCalls.size > 0 && (
        <div className="active-tools">
          {Array.from(activeToolCalls.entries()).map(([id, tool]) => (
            <ToolCallCard key={id} toolCall={tool} />
          ))}
        </div>
      )}

      {/* 待处理工具栈 */}
      {pendingToolsStack.length > 0 && (
        <div className="pending-tools">
          <span>Queued: {pendingToolsStack.join(' → ')}</span>
        </div>
      )}
    </div>
  );
}
'''

print(use_streaming)

## 7. useAgentContext Hook

In [ ]:
# 访问 AgentProvider 上下文

use_context = '''
import { useAgentContext } from '@memstack-agent-ui/react';

function ConnectionStatus() {
  const { status, isRunning, error, wsUrl } = useAgentContext();

  return (
    <div className="connection-status">
      <span className={`status-dot ${status}`} />
      <span>{status}</span>
      
      {isRunning && <span className="running-badge">Running</span>}
      {error && <span className="error">{error.message}</span>}
    </div>
  );
}

function SubmitButton() {
  const { submit, isRunning } = useAgentContext();

  return (
    <button
      onClick={() => submit('Hello!')}
      disabled={isRunning}
    >
      {isRunning ? 'Running...' : 'Send Hello'}
    </button>
  );
}
'''

print(use_context)

## 8. 完整聊天组件

In [ ]:
# 完整的聊天组件示例

complete_chat = '''
import { useState } from 'react';
import { useAgentChat, useConversation, useStreaming } from '@memstack-agent-ui/react';

interface ChatProps {
  conversationId: string;
}

export function Chat({ conversationId }: ChatProps) {
  const [input, setInput] = useState('');

  // 聊天 hook
  const { submit, isRunning, status, error } = useAgentChat({
    wsUrl: 'ws://localhost:8000/agent/ws',
    conversationId,
    onComplete: () => console.log('Done!'),
  });

  // 会话状态
  const { messages, agentState, costTracking } = useConversation({
    conversationId,
  });

  // 流式状态
  const {
    isStreaming,
    streamingAssistantContent,
    streamingThought,
    activeToolCalls,
  } = useStreaming({ conversationId });

  const handleSubmit = () => {
    if (input.trim() && !isRunning) {
      submit(input.trim());
      setInput('');
    }
  };

  return (
    <div className="chat-container">
      {/* 头部 */}
      <header className="chat-header">
        <h1>Agent Chat</h1>
        <div className="status">
          <span className={`dot ${status}`} />
          <span>{status}</span>
          {agentState !== 'idle' && <span> · {agentState}</span>}
          {costTracking && <span> · ${costTracking.totalCost.toFixed(4)}</span>}
        </div>
      </header>

      {/* 消息区域 */}
      <main className="chat-messages">
        {messages.map((msg) => (
          <MessageBubble key={msg.id} message={msg} />
        ))}

        {/* 流式思考 */}
        {streamingThought && (
          <div className="thinking">
            💭 {streamingThought}
          </div>
        )}

        {/* 流式响应 */}
        {streamingAssistantContent && (
          <div className="assistant streaming">
            {streamingAssistantContent}
          </div>
        )}

        {/* 活动工具 */}
        {activeToolCalls.size > 0 && (
          <div className="tools">
            {Array.from(activeToolCalls.values()).map((tool) => (
              <ToolBadge key={tool.callId} tool={tool} />
            ))}
          </div>
        )}
      </main>

      {/* 输入区域 */}
      <footer className="chat-input">
        <input
          value={input}
          onChange={(e) => setInput(e.target.value)}
          onKeyDown={(e) => e.key === 'Enter' && handleSubmit()}
          disabled={isRunning}
          placeholder="Type a message..."
        />
        <button onClick={handleSubmit} disabled={isRunning || !input.trim()}>
          {isStreaming ? '...' : 'Send'}
        </button>
        {error && <div className="error">{error.message}</div>}
      </footer>
    </div>
  );
}
'''

print(complete_chat)

## 9. 带文件附件

In [ ]:
# 支持文件附件

file_attachments = '''
import { useAgentChat } from '@memstack-agent-ui/react';

function ChatWithFiles() {
  const { submit, isRunning } = useAgentChat({
    wsUrl: 'ws://localhost:8000/agent/ws',
    conversationId: 'conv-123',
  });

  const [input, setInput] = useState('');
  const [attachments, setAttachments] = useState<string[]>([]);

  const handleFileUpload = async (file: File) => {
    // 上传文件到服务器
    const formData = new FormData();
    formData.append('file', file);
    
    const response = await fetch('/api/upload', {
      method: 'POST',
      body: formData,
    });
    
    const { fileId } = await response.json();
    setAttachments((prev) => [...prev, fileId]);
  };

  const handleSubmit = () => {
    if (input.trim()) {
      // 提交消息和文件附件
      submit(input.trim(), attachments.length > 0 ? attachments : undefined);
      setInput('');
      setAttachments([]);
    }
  };

  return (
    <div>
      <input
        value={input}
        onChange={(e) => setInput(e.target.value)}
      />
      
      <input
        type="file"
        onChange={(e) => e.target.files?.[0] && handleFileUpload(e.target.files[0])}
      />
      
      {attachments.length > 0 && (
        <div className="attachments">
          {attachments.length} file(s) attached
        </div>
      )}
      
      <button onClick={handleSubmit} disabled={isRunning}>
        Send
      </button>
    </div>
  );
}
'''

print(file_attachments)

## 总结

本 notebook 演示了 @memstack-agent-ui/react 的 Hooks：

1. **useAgentChat** - 主要聊天 hook，管理 WebSocket 和消息提交
2. **useConversation** - 访问完整会话状态（timeline、messages、tasks）
3. **useStreaming** - 访问流式内容（thought、assistant content、tool calls）
4. **useAgentContext** - 访问 AgentProvider 上下文
5. **AgentProvider** - 包装应用提供 WebSocket 连接

设计模式类似于 Vercel AI SDK 的 useChat hook。

下一步：查看 `fe-04-components.ipynb` 了解内置组件。